In [1]:
import numpy as np
import pandas as pd

In [2]:
import sys

try:
    from labjournal.core.databaseModel import *
except ImportError:
    sys.path.append("../")
    from labjournal.core.databaseModel import *

In [3]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from sqlalchemy import not_, or_, and_

In [4]:
# db_path = 'test_WORKS.db'
db_path = '/home/andrejb/Research/SIMS.db'

## For migration:

In [131]:
def listed(alist):
    return ",".join("{}".format(i) for i in alist)

In [6]:
def openDatabase(db_path):
    engine = create_engine('sqlite:///{}'.format(db_path))
    Session = sessionmaker(bind=engine)
    session = Session()
    return session

In [13]:
def getTags(db_path):
    s = openDatabase(db_path)
    q = s.query(Keywords).filter(Keywords.value == None)
    s.close()
    return [e.name for e in q.all()]

In [14]:
def getKeywords(db):
    s = openDatabase(db_path)
    q = s.query(Keywords)
    keywords = [e.name for e in q.filter(Keywords.value != None).all()]
    key_dict = {}
    for k in np.unique(keywords):
        key_dict[k] = np.unique([e.value for e in q.filter(Keywords.value != None, Keywords.name == k).all()])
    s.close()
    return key_dict

In [22]:
def getEntryTable(db_path):
    s = openDatabase(db_path)
    
    # get DB tables as pandas DataFrames
    main         = pd.read_sql_table("main", s.bind)[["id", "entry_id", "path"]]
    keywords_raw = pd.read_sql_table("keywords", s.bind)
    keywords     = keywords_raw[keywords_raw['value'].notna()]

    tags = keywords_raw[~keywords_raw['value'].notna()]
    tags = tags.drop('value',axis=1)\
                .groupby("main_id")\
                .agg({"name" : listed})\
                .rename(index=int, columns={"name": "tags"})
    s.close()
    
    # inner join to get the connection between entries and keywords
    m = pd.merge(main, keywords, left_on='id', right_on="main_id", how="inner")
    # pivot table reduces it to columns
    p = m.pivot(index='id_x', columns='name')["value"]
    # DataFrame where one can search by keyword and tags
    main_out = pd.concat([main.set_index('id'), p, tags], axis=1)
    return main_out

## HowTo

In [10]:
# db = openDatabase(db_path) # open database
# one should not keep the DB open

In [16]:
getTags(db_path)  # get all tags which are used in DB

[u'test', u'test', u'test2', u'test2']

In [18]:
getKeywords(db_path)  # get all keyword with the values they have

{u'chainlen': array([u'1', u'2'], dtype='<U1'),
 u'expansion_from': array([u'2016_08_09_2ub_k11_01', u'2016_08_09_2ub_k11_02',
        u'2016_08_09_2ub_k27_01', u'2016_08_09_2ub_k27_02',
        u'2016_08_09_2ub_k29_01', u'2016_08_09_2ub_k29_02',
        u'2016_08_09_2ub_k33_01', u'2016_08_09_2ub_k33_02',
        u'2016_08_09_2ub_k48_01', u'2016_08_09_2ub_k48_02',
        u'2016_08_09_2ub_k63_01', u'2016_08_09_2ub_k6_01',
        u'2016_08_09_2ub_k6_02', u'2016_08_10_2ub_m1_01',
        u'2016_08_16_2ub_m1_02', u'2016_11_26_2ub_k11_03',
        u'2016_11_26_2ub_k11_04', u'2016_11_26_2ub_k11_05',
        u'2016_11_26_2ub_k11_06', u'2016_11_26_2ub_k27_03',
        u'2016_11_26_2ub_k27_04', u'2016_11_26_2ub_k27_05',
        u'2016_11_26_2ub_k27_06', u'2016_11_26_2ub_k29_03',
        u'2016_11_26_2ub_k29_04', u'2016_11_26_2ub_k29_05',
        u'2016_11_26_2ub_k29_06', u'2016_11_26_2ub_k33_03',
        u'2016_11_26_2ub_k33_04', u'2016_11_26_2ub_k33_05',
        u'2016_11_26_2ub_k33_06', u'2

In [132]:
main_out = getEntryTable(db_path) # load whole table

In [24]:
main_out[main_out.tags.str.contains("test").notna()] # select by tag

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
124,2018_04_06_2xUb_unlinked_48,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,"test, test2"
132,2018_04_06_2xUb_unlinked_46,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,test2
585,2018_04_06_2xUb_unlinked_47,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,test


In [25]:
main_out[main_out["ff"] == "MARTINI"] # select by keyword

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
7,2018_04_06_2xUb_unlinked_22,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
9,2017_07_21_2ub_k11_02_02,/home/andrejb/Research/SIMS/2017_07_21_2ub_k11...,2,None,None,None,None,None,MARTINI,k11,...,None,02,None,UBQ,None,None,02,None,None,NaN
14,2016_08_09_2ub_k11_01,/home/andrejb/Research/SIMS/2016_08_09_2ub_k11_01,2,None,None,None,None,None,MARTINI,k11,...,None,01,None,UBQ,None,None,01,None,None,NaN
16,2017_04_26_M_ub1_300K,/home/andrejb/Research/SIMS/2017_04_26_M_ub1_300K,1,None,None,None,None,None,MARTINI,None,...,1,None,None,UBQ,None,None,None,None,300,NaN
17,2017_08_01_2ub_k48_01_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k48...,2,None,None,None,None,None,MARTINI,k48,...,None,06,None,UBQ,None,None,01,None,None,NaN
18,2017_08_01_2ub_k29_01_03,/home/andrejb/Research/SIMS/2017_08_01_2ub_k29...,2,None,None,None,None,None,MARTINI,k29,...,None,03,None,UBQ,None,None,01,None,None,NaN
21,2018_04_06_2xUb_unlinked_23,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
23,2017_08_01_2ub_k11_02_06,/home/andrejb/Research/SIMS/2017_08_01_2ub_k11...,2,None,None,None,None,None,MARTINI,k11,...,None,06,None,UBQ,None,None,02,None,None,NaN
24,2018_04_06_2xUb_unlinked_34,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
32,2016_11_26_2ub_k27_04,/home/andrejb/Research/SIMS/2016_11_26_2ub_k27_04,2,None,None,None,2016_08_09_2ub_k27_01,middle,MARTINI,k27,...,None,02,None,UBQ,None,None,None,extension,None,NaN


In [163]:
def selectByKeyword(table, name, value):
    # another way is to use:
    # main_out['ff'].isin(["GROMOS", "MARTINI"])
    return table[name] == value

def selectByTag(table, tag):
    split = table.tags.str.split(",")
    mask  = [True if np.any(np.array(i[1]) == tag) else False for i in split.iteritems()]
    return pd.Series(mask, index=range(1, len(mask) + 1))

In [165]:
# combinations of selections
ff       = selectByKeyword(main_out, "ff", "MARTINI")
chainlen = selectByKeyword(main_out, "chainlen", "1")
tags     = selectByTag(main_out, "test2")

main_out[ff & chainlen & tags]

,entry_id,path,chainlen,expansion_from,expansion_point,expansion_type,extension_from,extension_type,ff,linker,...,n_chains,number,pH,protein,seq,src,structure,stype,temp,tags
7,2018_04_06_2xUb_unlinked_22,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
16,2017_04_26_M_ub1_300K,/home/andrejb/Research/SIMS/2017_04_26_M_ub1_300K,1,None,None,None,None,None,MARTINI,None,...,1,None,None,UBQ,None,None,None,None,300,NaN
21,2018_04_06_2xUb_unlinked_23,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
24,2018_04_06_2xUb_unlinked_34,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
33,2018_02_08_2xUb_unlinked_2,/home/andrejb/Research/SIMS/2018_02_08_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
46,2018_04_06_2xUb_unlinked_29,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
81,2017_04_26_M_ub1_330K,/home/andrejb/Research/SIMS/2017_04_26_M_ub1_330K,1,None,None,None,None,None,MARTINI,None,...,1,None,None,UBQ,None,None,None,None,330,NaN
115,2018_04_06_2xUb_unlinked_18,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
122,2018_04_06_2xUb_unlinked_31,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN
143,2018_04_06_2xUb_unlinked_36,/home/andrejb/Research/SIMS/2018_04_06_2xUb_un...,1,None,None,None,None,None,MARTINI,None,...,2,None,None,UBQ,None,None,None,None,None,NaN


/bin/sh: 1: ff: not found


## Deva

Filter Simulations by:

* entry id
* tags
* keywords
* groups

for expressions:

* \*pattern*
* ! is not
* one or several by connected by and